# Laboratoria 4

In [21]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [22]:
# create dataset
X, y = make_classification(n_samples=1000, n_features=20, random_state=1, n_informative=10, n_redundant=10)

## Fixed folds crossvalidation

In [3]:
?GridSearchCV

In [8]:


X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])

X, y = make_classification(n_samples=1000, n_features=20, random_state=1, n_informative=10, n_redundant=10)

In [28]:
kf = KFold(n_splits=5)

In [29]:
kf.split(X)

<generator object _BaseKFold.split at 0x7fdd86f48270>

In [10]:
for train_index, test_index in kf.split(X):
    print('TRAIN: ', train_index, 'TEST:', test_index)

TRAIN:  [200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253
 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271
 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289
 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307
 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325
 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343
 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361
 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379
 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397
 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415
 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433
 434 435 436 437 438 439 440 441 442 443 44

## Nested resampling

In [15]:
cv_outer = KFold(n_splits = 5, random_state=123, shuffle=True)
space = dict()
space['n_estimators'] = [10, 100, 500]
space['max_features'] = [2, 4, 6]

In [16]:
outer_results=list()
for train_ix, test_ix in cv_outer.split(X):
    X_train = X[train_ix,:]
    X_test = X[test_ix,:]
    
    y_train = y[train_ix]
    y_test = y[test_ix]
    
    cv_inner= KFold(n_splits=3)
    
    model = RandomForestClassifier()
    
    search = GridSearchCV(model, space, scoring= 'accuracy', cv = cv_inner, refit = True)
    
    result = search.fit(X_train, y_train)
    
    best_model = result.best_estimator_
    
    outer_results.append(best_model)
    
    yhat = best_model.predict(X_test)
    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))

>acc=0.930, est=0.925, cfg={'max_features': 2, 'n_estimators': 500}
>acc=0.910, est=0.916, cfg={'max_features': 6, 'n_estimators': 100}
>acc=0.940, est=0.917, cfg={'max_features': 4, 'n_estimators': 500}
>acc=0.925, est=0.912, cfg={'max_features': 2, 'n_estimators': 500}
>acc=0.920, est=0.919, cfg={'max_features': 2, 'n_estimators': 500}


## Pipelines

Służą do tworzenia ciągu operacji i łączenia preprocessingu z trenowaniem modeli. Wszystkie operacje zdefiniowane w pipelinie będą dopasowywane (metoda `.fit`) na danych treningowych, a następnie nie zmieniając obliczonych parametrów wewnętrznych (metoda `.transform`).

In [25]:
X, y = make_classification(n_samples=1000, n_features=20, random_state=1, n_informative=10, n_redundant=10)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set
pipe.fit(X_train, y_train)
Pipeline(steps=[('scaler', StandardScaler()), ('svc', SVC())])
pipe.score(X_test, y_test)

0.972

### Pipeline + Grid/RandomSearch

Można zdefiniować przestrzeń przeszukiwań też dla metod preprcessingu.

In [27]:
# Pipeline + GridSearchCV
pca = PCA()
# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

# set the tolerance to a large value to make the example faster
logistic = LogisticRegression(max_iter=10000, tol=0.1)
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("logistic", logistic)])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    "pca__n_components": [2, 5, 10,15],
    "logistic__C": np.logspace(-4, 4, 4),
}
search = GridSearchCV(pipe, param_grid, n_jobs=2)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.901):
{'logistic__C': 21.54434690031882, 'pca__n_components': 10}
